# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('model.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 13758 = 89.41314096315071


# Treinamento

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0814 21:24:06.801036 140582074754880 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0814 21:24:06.901064 140582074754880 deprecation.py:323] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 276,746
Non-trainable params: 1,538,800
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 53s 5ms/step - loss: 1.2118 - categorical_accuracy: 0.5959 - val_loss: 0.9387 - val_categorical_accuracy: 0.6898
Epoch 2/20
10649/10649 [==============================] - 52s 5ms/step - loss: 0.6484 - categorical_accuracy: 0.7894 - val_loss: 0.6783 - val_categorical_accuracy: 0.7754

In [13]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 61s 6ms/step - loss: 1.1480 - categorical_accuracy: 0.6179 - val_loss: 0.8972 - val_categorical_accuracy: 0.7142
Epoch 2/20
10649/10649 [==============================] - 58s 5ms/step - loss: 0.5203 - categorical_accuracy: 0.8294 - val_loss: 0.6987 - val_categorical_accuracy: 0.7679
Epoch

In [22]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 256)               91392     
_________________________________________________________________
dense_11 (Dense)             (None, 10)                2570      
Total params: 1,632,762
Trainable params: 1,632,762
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 24s 2ms/step - loss: 1.6143 - categorical_accuracy: 0.4610 - val_loss: 1.5065 - val_categorical_accuracy: 0.5024
Epoch 2/20
10649/10649 [==============================] - 22s 2ms/step - loss: 1.2121 - categorical_accuracy: 0.5979 - val_loss: 1.5370 - val_categorical_accuracy: 0.5100
Epoch

In [15]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,906,938
Trainable params: 1,906,938
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 69s 7ms/step - loss: 1.1645 - categorical_accuracy: 0.6217 - val_loss: 1.2635 - val_categorical_accuracy: 0.6125
Epoch 2/20
10649/10649 [==============================] - 69s 6ms/step - loss: 0.6397 - categorical_accuracy: 0.7992 - val_loss: 0.7412 - val_categorical_accuracy: 0.7724
Epoch

In [16]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               182784    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 1,726,714
Trainable params: 1,726,714
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 35s 3ms/step - loss: 1.6490 - categorical_accuracy: 0.4494 - val_loss: 1.8254 - val_categorical_accuracy: 0.3954
Epoch 2/20
10649/10649 [==============================] - 35s 3ms/step - loss: 1.2326 - categorical_accuracy: 0.6004 - val_loss: 2.3334 - val_categorical_accuracy: 0.3526
Epoch

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               548352    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 2,092,282
Trainable params: 2,092,282
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 97s 9ms/step - loss: 1.1326 - categorical_accuracy: 0.6294 - val_loss: 0.8253 - val_categorical_accuracy: 0.7289
Epoch 2/20
10649/10649 [==============================] - 96s 9ms/step - loss: 0.5275 - categorical_accuracy: 0.8317 - val_loss: 0.5711 - val_categorical_accuracy: 0.8239
Epoch

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 2,275,066
Trainable params: 2,275,066
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 127s 12ms/step - loss: 1.2238 - categorical_accuracy: 0.5926 - val_loss: 1.0035 - val_categorical_accuracy: 0.6748
Epoch 2/20
10649/10649 [==============================] - 126s 12ms/step - loss: 0.6745 - categorical_accuracy: 0.7864 - val_loss: 0.8356 - val_categorical_accuracy: 0.7356
E

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 1,628,026
Trainable params: 1,628,026
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.1807 - categorical_accuracy: 0.6059 - val_loss: 1.0647 - val_categorical_accuracy: 0.6526
Epoch 2/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.5818 - categorical_accuracy: 0.8126 - val_loss: 0.6682 - val_categorical_accuracy: 0.7890
Epoch

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                31680     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 1,571,130
Trainable params: 1,571,130
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.2914 - categorical_accuracy: 0.5668 - val_loss: 1.0646 - val_categorical_accuracy: 0.6549
Epoch 2/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.7155 - categorical_accuracy: 0.7705 - val_loss: 0.7722 - val_categorical_accuracy: 0.7540
Epoch

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_6 (GRU)                  (None, 512)               941568    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 2,485,498
Trainable params: 2,485,498
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 59s 6ms/step - loss: 1.1489 - categorical_accuracy: 0.6313 - val_loss: 0.7766 - val_categorical_accuracy: 0.7503
Epoch 2/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.4761 - categorical_accuracy: 0.8431 - val_loss: 0.6214 - val_categorical_accuracy: 0.8040
Epoch

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_12 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 1.1634 - categorical_accuracy: 0.6098 - val_loss: 0.8392 - val_categorical_accuracy: 0.7293
Epoch 2/20
10649/10649 [====================

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_14 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_15 (Dense)             (None, 10)                650       
Total params: 1,830,074
Trainable params: 1,830,074
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 60s 6ms/step - loss: 1.1905 - categorical_accuracy: 0.6094 - val_loss: 0.9209 - val_categorical_accuracy: 0.7101
Epoch 2/20
10649/10649 [====================

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_16 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_17 (Dense)             (None, 10)                330       
Total params: 1,821,530
Trainable params: 1,821,530
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 58s 5ms/step - loss: 1.2168 - categorical_accuracy: 0.5940 - val_loss: 0.8186 - val_categorical_accuracy: 0.7360
Epoch 2/20
10649/10649 [====================

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0815 02:04:12.279198 140582074754880 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 66s 6ms/step - loss: 1.2637 - categorical_accuracy: 0.5808 - val_loss: 0.9423 - val_categorical_accuracy: 0.6951
Epoch 2/20
10649/10649 [==============================] - 64s 6ms/step - loss: 0.6337 - categorical_accuracy: 0.7922 - val_loss: 0.6923 - val_categorical_accuracy: 0.7743
Epoch

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 65s 6ms/step - loss: 1.3159 - categorical_accuracy: 0.5654 - val_loss: 1.0998 - val_categorical_accuracy: 0.6466
Epoch 2/20
10649/10649 [==============================] - 63s 6ms/step - loss: 0.6600 - categorical_accuracy: 0.7855 - val_loss: 0.7376 - val_categorical_accuracy: 0.7570
Epoch

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.1139 - categorical_accuracy: 0.6325 - val_loss: 0.8058 - val_categorical_accuracy: 0.7469
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.4618 - categorical_accuracy: 0.8563 - val_loss: 0.5444 - val_categorical_accuracy: 0.8299
Epoch

In [29]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_21 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 57s 5ms/step - loss: 1.2950 - categorical_accuracy: 0.5663 - val_loss: 1.1309 - val_categorical_accuracy: 0.6125
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.6661 - categorical_accuracy: 0.7845 - val_loss: 0.7309 - val_categorical_accuracy: 0.7548
Epoch

In [30]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_14 (GRU)                 (None, 200, 256)          274176    
_________________________________________________________________
gru_15 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_22 (Dense)             (None, 10)                330       
Total params: 1,841,050
Trainable params: 1,841,050
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 111s 10ms/step - loss: 1.1626 - categorical_accuracy: 0.6144 - val_loss: 0.9095 - val_categorical_accuracy: 0.7116
Epoch 2/20
10649/10649 [==================

In [31]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_23 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 66s 6ms/step - loss: 1.44

In [32]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_25 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 30s 3ms/step - loss: 1.2959 - categorical_accuracy: 0.5693 - val_loss: 0.9541 - val_categorical_accuracy: 0.6793
Epoch 2/20
10649/10649 [==============================] - 28s 3ms/step - loss: 0.6356 - categorical_accuracy: 0.7932 - val_loss: 0.7265 - val_categorical_accuracy: 0.7627
Epoch